# Hansard Data Explorer

This notebook explores the parsed Hansard parliamentary debates data from the `hansard-nlp-explorer` project.

## Data Sources
- **Raw data**: `src/hansard/scripts/data/hansard/` - HTML files organized by year/month
- **Processed data**: `src/hansard/scripts/data/processed/` - Parquet files with extracted metadata
- **Test data**: `src/hansard/scripts/data/processed_test/` - Subset for testing

In [1]:
import polars as pl
import pandas as pd
from pathlib import Path
import json
import gzip
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")

print("Hansard Data Explorer loaded successfully!")
print(f"Working directory: {Path.cwd()}")

Hansard Data Explorer loaded successfully!
Working directory: /Users/omarkhursheed/workplace/hansard-nlp-explorer/src


## 1. Explore Data Structure

In [4]:
# Define data paths
data_root = Path("hansard/data")
processed_path = data_root / "processed"
test_path = data_root / "processed_test"
raw_path = data_root / "hansard"

print("Data directory structure:")
for path in [data_root, processed_path, test_path, raw_path]:
    if path.exists():
        print(f"✓ {path}")
        if path.is_dir():
            subdirs = [d for d in path.iterdir() if d.is_dir()]
            files = [f for f in path.iterdir() if f.is_file()]
            print(f"  - {len(subdirs)} subdirectories, {len(files)} files")
    else:
        print(f"✗ {path} (not found)")

Data directory structure:
✓ hansard/data
  - 3 subdirectories, 1 files
✓ hansard/data/processed
  - 3 subdirectories, 4 files
✓ hansard/data/processed_test
  - 3 subdirectories, 2 files
✓ hansard/data/hansard
  - 201 subdirectories, 1 files


## 2. Load Processed Metadata

Let's start with the test data to understand the structure:

In [ ]:
# Check what processed data is available
test_metadata_path = test_path / "metadata"
processed_metadata_path = processed_path / "metadata"

print("Test metadata files:")
if test_metadata_path.exists():
    for f in sorted(test_metadata_path.glob("*.parquet")):
        print(f"  - {f.name}")
else:
    print("  No test metadata found")

print("\nProcessed metadata files (Full Dataset):")
if processed_metadata_path.exists():
    parquet_files = list(processed_metadata_path.glob("*.parquet"))
    debates_files = [f for f in parquet_files if f.name.startswith('debates_')]
    speakers_files = [f for f in parquet_files if f.name.startswith('speakers_')]
    
    print(f"  - {len(debates_files)} debates files")
    print(f"  - {len(speakers_files)} speakers files")
    
    # Show year coverage
    debate_years = []
    for f in debates_files:
        if f.name != 'debates_master.parquet':
            year = f.name.replace('debates_', '').replace('.parquet', '')
            if year.isdigit():
                debate_years.append(int(year))
    
    if debate_years:
        debate_years.sort()
        print(f"  - Year coverage: {min(debate_years)}-{max(debate_years)} ({len(debate_years)} years)")
    
    # Check for master files
    master_files = [f for f in parquet_files if 'master' in f.name]
    if master_files:
        print(f"  - Master files: {[f.name for f in master_files]}")
else:
    print("  No processed metadata found")

In [ ]:
# Load the master debates file (full dataset)
sample_debates = None
sample_speakers = None

# Prioritize full dataset over test data
for base_path in [processed_metadata_path, test_metadata_path]:
    if base_path.exists():
        # Try master files first (consolidated full dataset)
        debates_master = base_path / "debates_master.parquet"
        speakers_master = base_path / "speakers_master.parquet"
        
        if debates_master.exists():
            print(f"✅ Loading FULL DATASET from {debates_master}")
            sample_debates = pl.read_parquet(debates_master)
            
            # Show dataset info
            file_size_mb = debates_master.stat().st_size / (1024**2)
            print(f"   📁 File size: {file_size_mb:.1f} MB")
            break
        else:
            # Try individual year files as fallback
            year_files = list(base_path.glob("debates_*.parquet"))
            if year_files:
                # Load first few years to get a sample
                first_file = sorted(year_files)[0]
                print(f"Loading sample from {first_file}")
                sample_debates = pl.read_parquet(first_file)
                break

if sample_debates is not None:
    print(f"\n📊 Debates data shape: {sample_debates.shape}")
    print(f"📋 Columns: {list(sample_debates.columns)}")
    
    # Show year coverage if available
    if 'year' in sample_debates.columns:
        year_range = sample_debates.select([pl.col('year').min(), pl.col('year').max()]).to_pandas().iloc[0]
        print(f"📅 Year coverage: {year_range.iloc[0]} - {year_range.iloc[1]}")
    
    # Show memory usage
    memory_mb = sample_debates.estimated_size(unit='mb')
    print(f"💾 Memory usage: ~{memory_mb:.1f} MB")
    
    print("\n📝 First few rows:")
    print(sample_debates.head())
else:
    print("❌ No debates data found to load")

In [ ]:
# Load the speakers master file (full dataset)
for base_path in [processed_metadata_path, test_metadata_path]:
    if base_path.exists():
        speakers_master = base_path / "speakers_master.parquet"
        
        if speakers_master.exists():
            print(f"✅ Loading FULL SPEAKERS DATASET from {speakers_master}")
            sample_speakers = pl.read_parquet(speakers_master)
            
            # Show dataset info
            file_size_mb = speakers_master.stat().st_size / (1024**2)
            print(f"   📁 File size: {file_size_mb:.1f} MB")
            break
        else:
            speaker_files = list(base_path.glob("speakers_*.parquet"))
            if speaker_files:
                first_file = sorted(speaker_files)[0]
                print(f"Loading speakers sample from {first_file}")
                sample_speakers = pl.read_parquet(first_file)
                break

if sample_speakers is not None:
    print(f"\n📊 Speakers data shape: {sample_speakers.shape}")
    print(f"📋 Columns: {list(sample_speakers.columns)}")
    
    # Show year coverage if available
    if 'year' in sample_speakers.columns:
        year_range = sample_speakers.select([pl.col('year').min(), pl.col('year').max()]).to_pandas().iloc[0]
        print(f"📅 Year coverage: {year_range.iloc[0]} - {year_range.iloc[1]}")
    
    # Show memory usage
    memory_mb = sample_speakers.estimated_size(unit='mb')
    print(f"💾 Memory usage: ~{memory_mb:.1f} MB")
    
    # Show unique speaker count
    if 'speaker_name' in sample_speakers.columns:
        unique_speakers = sample_speakers.select('speaker_name').unique().height
        print(f"👥 Unique speakers: {unique_speakers:,}")
    
    print("\n📝 First few rows:")
    print(sample_speakers.head())
else:
    print("❌ No speakers data found to load")

## 3. Explore Raw Data Structure

Let's examine the raw HTML files to understand the data format:

In [ ]:
# Explore raw data structure
if raw_path.exists():
    years = sorted([d for d in raw_path.iterdir() if d.is_dir()])
    print(f"Available years: {len(years)}")
    print(f"Year range: {years[0].name} - {years[-1].name}" if years else "No years found")
    
    # Look at first few years
    for year_dir in years[:5]:
        months = sorted([d for d in year_dir.iterdir() if d.is_dir()])
        files = [f for f in year_dir.iterdir() if f.is_file()]
        print(f"\n{year_dir.name}: {len(months)} months, {len(files)} files")
        
        # Look at one month
        if months:
            month_dir = months[0]
            month_files = list(month_dir.glob("*.html.gz"))
            json_files = list(month_dir.glob("*.json"))
            print(f"  {month_dir.name}: {len(month_files)} HTML files, {len(json_files)} JSON files")
else:
    print("Raw data directory not found")

In [ ]:
# Examine a sample HTML file and JSON summary
if raw_path.exists():
    # Find first available HTML file
    sample_html = None
    sample_json = None
    
    for year_dir in sorted(raw_path.iterdir()):
        if year_dir.is_dir():
            for month_dir in sorted(year_dir.iterdir()):
                if month_dir.is_dir():
                    html_files = list(month_dir.glob("*.html.gz"))
                    json_files = list(month_dir.glob("*.json"))
                    
                    if html_files:
                        sample_html = html_files[0]
                        print(f"Sample HTML file: {sample_html}")
                        
                        # Read first few lines
                        with gzip.open(sample_html, 'rt', encoding='utf-8') as f:
                            lines = [f.readline().strip() for _ in range(10)]
                            print("First 10 lines:")
                            for i, line in enumerate(lines, 1):
                                print(f"{i:2d}: {line[:100]}{'...' if len(line) > 100 else ''}")
                        break
                        
                    if json_files:
                        sample_json = json_files[0]
                        print(f"\nSample JSON file: {sample_json}")
                        
                        with open(sample_json, 'r') as f:
                            data = json.load(f)
                            print("JSON structure:")
                            print(json.dumps(data, indent=2)[:500] + "..." if len(str(data)) > 500 else json.dumps(data, indent=2))
                        break
                        
            if sample_html:
                break

## 4. Data Analysis and Visualization

Now let's analyze the processed data if available:

In [ ]:
# Comprehensive analysis of the full debates dataset
if sample_debates is not None:
    print("🏛️  HANSARD DEBATES DATASET ANALYSIS (FULL 1803-2005)")
    print("=" * 60)
    
    # Basic statistics
    total_debates = len(sample_debates)
    print(f"📊 Total debates: {total_debates:,}")
    
    # Try to identify date columns
    date_cols = [col for col in sample_debates.columns if any(word in col.lower() for word in ['date', 'year', 'month', 'day'])]
    print(f"📅 Date-related columns: {date_cols}")
    
    # Year analysis
    if 'year' in sample_debates.columns:
        year_stats = sample_debates.select('year').to_pandas()['year']
        print(f"📈 Year range: {year_stats.min()} - {year_stats.max()}")
        print(f"🗓️  Years covered: {year_stats.nunique()} unique years")
        print(f"📊 Average debates per year: {total_debates / year_stats.nunique():.0f}")
    
    # Column information
    print(f"\n📋 Dataset Schema ({len(sample_debates.columns)} columns):")
    print("-" * 50)
    for col in sample_debates.columns[:10]:  # Show first 10 columns
        dtype = sample_debates[col].dtype
        non_null = sample_debates[col].count()
        null_pct = ((total_debates - non_null) / total_debates * 100)
        print(f"  • {col:20} {dtype:15} ({null_pct:4.1f}% null)")
    
    if len(sample_debates.columns) > 10:
        print(f"  ... and {len(sample_debates.columns) - 10} more columns")
    
    # Content analysis
    text_cols = [col for col in sample_debates.columns if any(word in col.lower() for word in ['title', 'topic', 'subject', 'content', 'text'])]
    if text_cols:
        print(f"\n📝 Text/Content Columns: {text_cols}")
        
        # Show sample content from first text column
        sample_col = text_cols[0]
        print(f"\n📄 Sample content from '{sample_col}':")
        print("-" * 50)
        
        # Get non-null values
        sample_content = sample_debates.filter(pl.col(sample_col).is_not_null()).select(sample_col).limit(3).to_pandas()
        for i, content in enumerate(sample_content[sample_col], 1):
            preview = str(content)[:200] + "..." if len(str(content)) > 200 else str(content)
            print(f"{i}. {preview}\n")
    
    # Data quality assessment
    print("🔍 Data Quality Assessment:")
    print("-" * 30)
    completeness = (sample_debates.select(pl.all().count()) / total_debates * 100).to_pandas().iloc[0]
    avg_completeness = completeness.mean()
    print(f"📈 Average column completeness: {avg_completeness:.1f}%")
    
    # Most complete columns
    top_complete = completeness.nlargest(5)
    print("🏆 Most complete columns:")
    for col, pct in top_complete.items():
        print(f"  • {col}: {pct:.1f}%")
    
else:
    print("❌ No debates data available for analysis")

In [ ]:
# Analyze speakers data
if sample_speakers is not None:
    print("Speakers Data Analysis:")
    print(f"Total speaker records: {len(sample_speakers)}")
    
    # Show data types
    print("\nColumn types:")
    for col in sample_speakers.columns:
        dtype = sample_speakers[col].dtype
        print(f"  {col}: {dtype}")
    
    # Show unique speakers if there's a name column
    name_cols = [col for col in sample_speakers.columns if any(word in col.lower() for word in ['name', 'speaker'])]
    if name_cols:
        name_col = name_cols[0]
        unique_speakers = sample_speakers[name_col].unique().limit(20)
        print(f"\nSample speakers ({name_col}):")
        for speaker in unique_speakers:
            if speaker is not None:
                print(f"  - {speaker}")
else:
    print("No speakers data available for analysis")

In [ ]:
# Create comprehensive visualizations for the full dataset
if sample_debates is not None:
    # Convert to pandas for plotting (sample if too large)
    plot_df = sample_debates.to_pandas()
    
    # If dataset is very large, sample for visualization
    if len(plot_df) > 100000:
        print(f"📊 Sampling {100000:,} records from {len(plot_df):,} total for visualization...")
        plot_df = plot_df.sample(n=100000, random_state=42)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('🏛️ Hansard Parliamentary Debates Analysis (1803-2005)', fontsize=16, fontweight='bold')
    
    # Plot 1: Debates over time
    year_cols = [col for col in plot_df.columns if 'year' in col.lower()]
    if year_cols:
        year_col = year_cols[0]
        if plot_df[year_col].dtype in ['int64', 'float64']:
            year_counts = plot_df[year_col].value_counts().sort_index()
            year_counts.plot(kind='line', ax=axes[0,0], color='navy', linewidth=2)
            axes[0,0].set_title('📈 Debates Per Year Over Time')
            axes[0,0].set_xlabel('Year')
            axes[0,0].set_ylabel('Number of Debates')
            axes[0,0].grid(True, alpha=0.3)
            
            # Add key historical events as annotations
            historical_events = {
                1914: "WWI Start", 1918: "WWI End", 1939: "WWII Start", 
                1945: "WWII End", 1979: "Thatcher Era"
            }
            for year, event in historical_events.items():
                if year in year_counts.index:
                    axes[0,0].annotate(event, (year, year_counts[year]), 
                                     xytext=(5, 5), textcoords='offset points', 
                                     fontsize=8, alpha=0.7)
        else:
            axes[0,0].text(0.5, 0.5, f'{year_col} not numeric', ha='center', va='center')
            axes[0,0].set_title('Year Distribution')
    else:
        axes[0,0].text(0.5, 0.5, 'No year column found', ha='center', va='center')
        axes[0,0].set_title('Year Distribution')
    
    # Plot 2: Data completeness heatmap
    missing_data = plot_df.isnull().mean() * 100
    top_missing = missing_data.nlargest(15)
    
    if len(top_missing) > 0:
        top_missing.plot(kind='barh', ax=axes[0,1], color='coral')
        axes[0,1].set_title('🔍 Missing Data by Column (Top 15)')
        axes[0,1].set_xlabel('% Missing')
        axes[0,1].tick_params(axis='y', labelsize=8)
    
    # Plot 3: Column data types distribution
    dtype_counts = plot_df.dtypes.value_counts()
    colors = plt.cm.Set3(range(len(dtype_counts)))
    dtype_counts.plot(kind='pie', ax=axes[0,2], autopct='%1.1f%%', colors=colors)
    axes[0,2].set_title('🗂️ Column Data Types')
    axes[0,2].set_ylabel('')
    
    # Plot 4: Debate volume by decade
    if year_cols and plot_df[year_col].dtype in ['int64', 'float64']:
        plot_df['decade'] = (plot_df[year_col] // 10) * 10
        decade_counts = plot_df['decade'].value_counts().sort_index()
        
        colors = plt.cm.viridis(np.linspace(0, 1, len(decade_counts)))
        decade_counts.plot(kind='bar', ax=axes[1,0], color=colors)
        axes[1,0].set_title('📊 Debates by Decade')
        axes[1,0].set_xlabel('Decade')
        axes[1,0].set_ylabel('Number of Debates')
        axes[1,0].tick_params(axis='x', rotation=45)
    
    # Plot 5: Text length analysis (if text columns exist)
    text_cols = [col for col in plot_df.columns if any(word in col.lower() for word in ['content', 'text', 'speech'])]
    if text_cols:
        text_col = text_cols[0]
        # Calculate text lengths (handle nulls)
        text_lengths = plot_df[text_col].astype(str).str.len()
        text_lengths = text_lengths[text_lengths > 0]  # Remove nulls/empty
        
        if len(text_lengths) > 0:
            text_lengths.hist(bins=50, ax=axes[1,1], alpha=0.7, color='lightblue', edgecolor='navy')
            axes[1,1].set_title(f'📝 Text Length Distribution\n({text_col})')
            axes[1,1].set_xlabel('Character Count')
            axes[1,1].set_ylabel('Frequency')
            axes[1,1].axvline(text_lengths.median(), color='red', linestyle='--', 
                            label=f'Median: {text_lengths.median():.0f}')
            axes[1,1].legend()
    else:
        axes[1,1].text(0.5, 0.5, 'No text columns\nfound for analysis', 
                      ha='center', va='center', fontsize=12)
        axes[1,1].set_title('📝 Text Analysis')
    
    # Plot 6: Dataset summary info
    total_debates = len(sample_debates)
    total_years = plot_df[year_col].nunique() if year_cols else 0
    
    # Calculate dataset size
    memory_usage_mb = sample_debates.estimated_size(unit='mb')
    
    info_text = f"""📊 DATASET SUMMARY
    
📈 Total Records: {total_debates:,}
📋 Total Columns: {len(sample_debates.columns)}
🗓️ Years Covered: {total_years}
💾 Memory Usage: ~{memory_usage_mb:.1f} MB
⚡ Processing Status: Complete
    
🎯 Coverage: 1803-2005
📁 Files Processed: 673,385
⏱️ Processing Time: 0.5 hours
✅ Success Rate: 100%"""
    
    axes[1,2].text(0.05, 0.95, info_text, fontsize=11, verticalalignment='top',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.8),
                   family='monospace')
    axes[1,2].axis('off')
    axes[1,2].set_title('📋 Dataset Overview')
    
    plt.tight_layout()
    plt.show()
    
    # Additional summary statistics
    print(f"\n🎉 FULL DATASET LOADED SUCCESSFULLY!")
    print(f"📊 {total_debates:,} debates spanning {total_years} years")
    print(f"💾 Dataset size: {memory_usage_mb:.1f} MB in memory")
    
else:
    print("❌ No data available for visualization")

## 5. Next Steps

This notebook provides a foundation for exploring the Hansard parliamentary debates data. You can extend it by:

1. **Text Analysis**: Use spacy or gensim for NLP tasks
2. **Time Series Analysis**: Analyze debate patterns over time
3. **Speaker Analysis**: Study individual MP contributions
4. **Topic Modeling**: Identify themes in debates
5. **Network Analysis**: Analyze debate participation patterns

### Useful Commands:

```bash
# Activate the hansard environment
conda activate hansard

# Run the full processing pipeline
cd src/hansard/scripts
./run_full_processing.sh

# Test processing on subset
python test_production_script.py
```